# Feature Selection (BorutaShap)

## Data importing

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
# Map the old name to the new function
scipy.stats.binom_test = scipy.stats.binomtest

# Now import BorutaShap
from BorutaShap import BorutaShap, load_data

In [14]:
from pathlib import Path
dataset=pd.read_csv("dataset_final_feature_engineered.csv")
dataset.head(3)

,fraud_bool,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,proposed_credit_limit,foreign_request,device_os,keep_alive_session,device_distinct_emails_8w,month,...,phone_home_and_mobile_valid,proposed_credit_limit_binned,prev_address_months_count_binary,banking_relationship_stability,income_level,customer_age_binned,payment_type_engineered,employment_status_engineered,intended_balcon_amount_binned_engineered,bank_branch_count_8w_binned_engineered
0,0,5,163,1,1500.0,0,linux,1,1,0,...,Invalid_Valid,1000+,prev_address_missing,bank_months_valid_has_other_cards_invalid,<0.9,<=40 (less fraud),AA,CB,52+,<=40
1,0,18,154,1,1500.0,0,other,1,1,0,...,Valid_Valid,1000+,prev_address_missing,bank_months_valid_has_other_cards_invalid,<0.9,<=40 (less fraud),Other,CA,Missing/Negative,<=40
2,0,11,89,1,200.0,0,windows,0,1,0,...,Invalid_Valid,<=200,prev_address_valid,bank_months_valid_has_other_cards_invalid,<0.9,<=40 (less fraud),Other,CA,Missing/Negative,<=40


## Feature Selection using BorutaShap

In [23]:
dataset.iloc[:,1:]

,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,proposed_credit_limit,foreign_request,device_os,keep_alive_session,device_distinct_emails_8w,month,name_email_similarity_logit,...,phone_home_and_mobile_valid,proposed_credit_limit_binned,prev_address_months_count_binary,banking_relationship_stability,income_level,customer_age_binned,payment_type_engineered,employment_status_engineered,intended_balcon_amount_binned_engineered,bank_branch_count_8w_binned_engineered
0,5,163,1,1500.0,0,linux,1,1,0,4.291948,...,Invalid_Valid,1000+,prev_address_missing,bank_months_valid_has_other_cards_invalid,<0.9,<=40 (less fraud),AA,CB,52+,<=40
1,18,154,1,1500.0,0,other,1,1,0,0.478637,...,Valid_Valid,1000+,prev_address_missing,bank_months_valid_has_other_cards_invalid,<0.9,<=40 (less fraud),Other,CA,Missing/Negative,<=40
2,11,89,1,200.0,0,windows,0,1,0,5.712664,...,Invalid_Valid,<=200,prev_address_valid,bank_months_valid_has_other_cards_invalid,<0.9,<=40 (less fraud),Other,CA,Missing/Negative,<=40
3,13,90,1,200.0,0,linux,1,1,0,-0.099683,...,Invalid_Valid,<=200,prev_address_valid,bank_months_valid_has_other_cards_invalid,<0.9,<=40 (less fraud),Other,CA,Missing/Negative,<=40
4,6,91,0,200.0,0,other,0,1,0,1.675493,...,Valid_Valid,<=200,prev_address_missing,bank_months_valid_has_other_cards_invalid,0.9,<=40 (less fraud),AA,CA,20-52,<=40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,8,305,1,1500.0,0,other,0,1,7,-1.948744,...,Valid_Valid,1000+,prev_address_missing,bank_months_valid_has_other_cards_invalid,<0.9,<=40 (less fraud),Other,CA,Missing/Negative,>40
999996,5,235,0,1000.0,0,macintosh,0,1,7,1.547444,...,Valid_Valid,200<x<=1000,prev_address_missing,bank_months_missing_has_other_cards_valid,0.9,<=40 (less fraud),AC,CA,0-20,<=40
999997,3,195,1,200.0,0,other,0,1,7,-1.807906,...,Invalid_Valid,<=200,prev_address_missing,bank_months_valid_has_other_cards_invalid,<0.9,<=40 (less fraud),AA,CA,20-52,<=40
999998,8,148,0,200.0,0,linux,1,1,7,-5.996917,...,Invalid_Valid,<=200,prev_address_valid,bank_months_valid_has_other_cards_invalid,0.9,<=40 (less fraud),Other,CA,Missing/Negative,<=40


In [24]:
# X is the entire dataframe without first column, while first column is y
X,y=dataset.iloc[:,1:], dataset.iloc[:,0]

In [27]:
# no model selected default is Random Forest, if classification is True it is a Classification problem
Feature_Selector = BorutaShap(importance_measure='shap',
                              classification=True)

'''
Sample: Boolean
	if true then a rowise sample of the data will be used to calculate the feature importance values

sample_fraction: float
	The sample fraction of the original data used in calculating the feature importance values only
        used if Sample==True.

train_or_test: string
	Decides whether the feature improtance should be calculated on out of sample data see the dicussion here.
        https://slds-lmu.github.io/iml_methods_limitations/pfi-data.html

normalize: boolean
            if true the importance values will be normalized using the z-score formula

verbose: Boolean
	a flag indicator to print out all the rejected or accepted features.
'''
Feature_Selector.fit(X=X, y=y, n_trials=100, sample=False,
            	      normalize=True,
		     verbose=True)

  0%|          | 0/100 [00:00<?, ?it/s]

ValueError: could not convert string to float: 'other'